<a href="https://colab.research.google.com/github/voxmenthe/BASHTOOLS/blob/master/Quick_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [0]:
torch.__version__

'1.4.0'

In [0]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [0]:
!ls "/content/gdrive/My Drive/Colab Notebooks"

 05_overfit_underfit.ipynb
'Copy of 01.01-Help-And-Documentation.ipynb'
'Copy of bag-of-tricks.ipynb'
'Copy of Captcha_cracker.ipynb'
'Copy of gpt-2-playground_.ipynb'
'Copy of hanet-tf2.ipynb'
'Copy of position_encoding.ipynb'
'Copy of transformer.ipynb'
'Copy of Youtube Recommender example.ipynb'
 PyTorch_RNN.ipynb
 TF_Class
'tf.keras for Researchers: Crash Course.ipynb'
 TPU_test_1.ipynb
'TransferLearn Tutorial NAACL 2019 JeffCopy'
 Untitled0.ipynb


### N-Gram Language Modeling

In an n-gram language model, given a sequence of words $w$, we weant to compute:
$$ P(w_i | w_{i-1}, w_{i-2}, ..., w_{i-n+1})$$
Where $w_i$ is the the ith word in the sequence

In [0]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()


# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [0]:
vocab = set(test_sentence)
len(vocab)

97

In [0]:
class NGramLanguageModeler(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super(NGramLanguageModeler, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(context_size * embedding_dim, 128)
    self.linear2 = nn.Linear(128, vocab_size)

  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1, -1))
    out = F.relu(self.linear1(embeds))
    out = self.linear2(out)
    log_probs = F.log_softmax(out, dims=1)
    return log_probs

  losses = []
  loss_function = nn.NLLLoss()
  model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
  optimizer = optim.SGD(model.parameters(), lr=0.001)

  for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
      # Step 1. Prepare the inputs to be passed to the model
      # (i.e. turn the words into integer indices and wrap them in tensors)
      context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

      # Step 2. Recall that torch *accumulates* gradients.
      # Before passing in a new instance, you need to zero ou the gradients from the old instance
      model.zero_grad()

      # Step 3. Run the forward pass, getting log probs over the next words
      log_probs = model(context_idxs)

      # Step 4. Compute your loss function. (Again, Torch wants the target word wrapped in a tensor)
      loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

      # Step 5.